# Word embeddings

In [ ]:
!pip install -U gensim
!pip install -U "datasets<3.0"

In [ ]:
import datasets # Biblioteca de manejo de conjuntos de datos para procesamiento de lenguaje natural
import re
import gensim # Biblioteca de modelamiento de lenguaje
import numpy as np # Biblioteca de manejo de datos vectoriales
import sklearn.linear_model # Módulo de sklearn de modelamiento lineal
import sklearn.decomposition # Módulo de sklearn donde está PCA
import plotly.express as px # Biblioteca de visualización
import urllib # Módulo de python para el manejo de consultas web

Cargamos el conjunto de datos del curso.

In [ ]:
spanish_diagnostics = datasets.load_dataset('fvillena/spanish_diagnostics') # Cargamos las particiones de entrenamiento y prueba

Preprocesamos el corpus.

In [ ]:
#Reutilizamos la función de normalización vista en labs anteriores
def normalize(text, remove_tildes = True):
    """Normaliza una cadena de texto convirtiéndo todo a minúsculas, quitando los caracteres no alfabéticos y los tildes"""
    text = text.lower() # Llevamos todo a minúscula
    text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text) # Reemplazamos los caracteres no alfabéticos por un espacio
    if remove_tildes:
        text = re.sub('á', 'a', text) # Reemplazamos los tildes
        text = re.sub('é', 'e', text)
        text = re.sub('í', 'i', text)
        text = re.sub('ó', 'o', text)
        text = re.sub('ú', 'u', text)
    return text

In [ ]:
spanish_diagnostics_normalized = spanish_diagnostics.map(
    lambda x: {
        "normalized_text" : normalize(x["text"])
    })

In [ ]:
spanish_diagnostics_normalized_tokenized = spanish_diagnostics_normalized.map(
    lambda x: {
        "tokenized_text" : x["normalized_text"].split()
    })

## Word embeddings

Los word embeddings son representaciones vectoriales de palabras. A cada una de las palabras de nuestro vocabulario se les asigna un vector que la representa y estas representaciones guardan relaciones semánticas del lenguaje natural. Por ejemplo, palabras semánticamente similares tienen a estar más cerca en el espacio que palabras no tan similares entre sí.

Para calcular estas representaciones utilizaremos el método Word2Vec, en su implementación de la biblioteca Gensim.

Para calcular las representaciones, simplemente pasamos nuestro corpus tokenizado al objeto Word2Vec.

In [ ]:
model = gensim.models.Word2Vec(spanish_diagnostics_normalized_tokenized["train"]["tokenized_text"])

Tenemos una representación de 100 dimensiones para cada una de las palabras del vocabulario.

In [ ]:
model.wv.vectors.shape

Si extraemos el vector asociado a una palabra se observa que está constituido de números reales.

In [ ]:
model.wv["pieza"]

Una de las métricas más utilizadas para establecer distancias entre vectores de palabras es la similaridad coseno, esta métrica está por defecto utilizada cuando deseamos saber las palabras más cercanas a un palabra que consultamos. Se observa que las palabras más cercanas a la palabra consultada guardan una estrecha relación semántica.

### Similaridad

In [ ]:
model.wv.most_similar("pieza")

Para continuar con la verificación de la similaridad de palabras establecemos pares de palabras en donde cada vez iremos alejándolas semánticamente. También podemos concluir que nuestro modelo pudo satisfactoriamente representar estas distancias de manera correcta.

In [ ]:
pairs = [
    ('pieza', 'diente'), # Palabras muy cercanas semánticamente
    ('pieza', 'caries'),
    ('pieza', 'protesis'),
    ('pieza', 'hueso'),
    ('pieza', 'sangre') # Palabras muy lejanas semanticamente
]

In [ ]:
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, model.wv.similarity(w1, w2)))

Una de las aplicaciones de la similaridad de palabras es la detección de palabras fuera de contexto en una lista de palabras. En este caso el modelo detectó correctamente la palabra extraña.

In [ ]:
model.wv.doesnt_match(["diente","periodontitis","cirrosis","corona","lengua"])

### Analogía

> Analogía, significa comparación o relación entre varias cosas, razones o conceptos; comparar o relacionar dos o más seres u objetos a través de la razón; señalando características generales y particulares comunes que permiten justificar la existencia de una propiedad en uno, a partir de la existencia de dicha propiedad en los otros.
>
>En el aspecto lógico, permite comparar un objeto con otros, en sus semejanzas y en sus diferencias. Una analogía permite la deducción de un término desconocido a partir del análisis de la relación que se establece entre dos términos conocidos.
>
> Analogía - Wikipedia

Tomando en cuenta esta definición podemos construir analogías como las siguientes:

* *pieza* es a *caries* como *ojo* es a *catarata*
* *bacteria* es a *antibiótico* como *virus* es a *antiviral*

Si a estos cuartetos de palabras le quitamos una, podemos transformar estas analogías en preguntas de analogía y esta es una tarea en la cual los word embeddings típicamente son probados:

* *pieza* es a *caries* como *ojo* es a *¿?*
* *bacteria* es a *antibiótico* como *virus* es a *¿?*

La relación vectorial que podemos construir para resolver estas pruebas es la siguiente:

Siendo $w_1$, $w_2$, $w_3$, $w_4$ los vectores asociados a las 4 palabras de la analogía, se cumple que $w_2 - w_1 	\approx w_4 - w_3$, por lo que para resolver una prueba de analogía, la posición de la palabra incógnita $w_4$ es $w_2 - w_1 + w_3$.

Aquí vemos como nuestro modelo resuelve correctamente una prueba de analogía.

In [ ]:
w_1 = model.wv["pieza"]
w_2 = model.wv["caries"]
w_3 = model.wv["ojo"]
w_4 = w_2 - w_1 + w_3

model.wv.similar_by_vector(w_4)

También gensim implementa el método Word2Vec.most_similar() que nos ayuda a resolver en 1 línea esta operación.

In [ ]:
model.wv.most_similar(positive=["caries", "ojo"], negative=["pieza"])

### Visualización

Para poder visualizar nuestro espacio vectorial debemos proyectar los embeddings en 2 dimensiones, para lo cual utilizaremos un método de reducción de dimensionalidad

In [ ]:
projector = sklearn.decomposition.PCA(3)
vectors_2d = projector.fit_transform(model.wv.vectors)[:,:2]

Podemos explorar en un espacio bidimensional las relaciones de cada una de nuestras palabras.

In [ ]:
fig = px.scatter(
    x=vectors_2d[:,0],
    y=vectors_2d[:,1],
    text=model.wv.index_to_key
)
fig.update_traces(mode="markers")

### Comparación

Existen word embeddings precalculados disponibles libremente en internet. Utilizaremos como embedding de dominio general el Spanish Billion Words Corpus Embeddings y compararemos el rendimiento sobre pruebas clínicas de los 2 embeddings.

Cargamos los embeddings.

In [ ]:
!curl -L -o sbwce_slim.txt https://raw.githubusercontent.com/dccuchile/CC66Q/main/tutorials/data/sbwce_slim.txt

In [ ]:
sbwce = gensim.models.KeyedVectors.load_word2vec_format("sbwce_slim.txt")

Podemos observar que si probamos este modelo sobre palabras del dominio clínico, los resultados de estas pruebas son desfavorables.

In [ ]:
sbwce.most_similar("pieza")

In [ ]:
for w1, w2 in pairs:
    try:
        print('%r\t%r\t%.2f' % (w1, w2, sbwce.similarity(w1, w2)))
    except KeyError as e:
        print(e)

In [ ]:
sbwce.doesnt_match(["diente","periodontitis","cirrosis","corona","lengua"])

Probamos una analogía típica utilizada para verificar el rendimiento de los modelos de dominio general.

In [ ]:
sbwce.most_similar(positive=["mujer", "rey"], negative=["hombre"], topn=3)

In [ ]:
sbwce_2d = projector.fit_transform(sbwce.vectors)[:,:2]

In [ ]:
fig = px.scatter(
    x=sbwce_2d[:,0],
    y=sbwce_2d[:,1],
    text=list(sbwce.index_to_key)
)
fig.update_traces(mode="markers")

## Clasificación de textos

Definimos una función que nos retornará una representación de un documento desde representaciones de palabras.

In [ ]:
def to_vector(tokens,model):
    """ Receives a sentence string along with a word embedding model and
    returns the vector representation of the sentence"""
    vec = np.zeros(model.vectors.shape[1]) # creates an empty vector of 300 dimensions
    for word in tokens: # iterates over the sentence
        if word in model: # checks if the word is in the word embedding
            vec += model[word] # adds every word embedding to the vector
    if np.linalg.norm(vec) > 0:
        return vec / np.linalg.norm(vec) # divides the vector by their normal
    else:
        return vec

Utilizaremos la función definida anteriormente para obtener la representación vectorial de cada uno de los ejemplos de nuestro conjunto de datos.

In [ ]:
spanish_diagnostics_normalized_tokenized_vectorized = spanish_diagnostics_normalized_tokenized.map(
    lambda x: {
        "vectorized_text" : to_vector(x["tokenized_text"],model.wv)
    })

Exploramos el espacio y verificamos que existe una separación relativamente clara entre los documentos de las distintas clases.

In [ ]:
def plot_3d(x, labels):
    """Generates a 3D scatter plot."""
    fig = px.scatter_3d(
        x=x[:, 0],
        y=x[:, 1],
        z=x[:, 2],
        color=labels,
        opacity=0.5
    )
    fig.update_traces(mode="markers", marker={'size': 2})
    fig.show()

In [ ]:
examples_3d = projector.fit_transform(spanish_diagnostics_normalized_tokenized_vectorized["test"]["vectorized_text"])
labels = spanish_diagnostics_normalized_tokenized_vectorized["test"].features["label"].int2str(spanish_diagnostics_normalized_tokenized_vectorized["test"]["label"])

plot_3d(examples_3d , labels)

Instanciamos y ajustamos un modelo de regresión logística para clasificar nuestros textos y vemos que este método tiene un rendimiento significativamente bueno

In [ ]:
classifier = sklearn.linear_model.LogisticRegression(solver='liblinear')

In [ ]:
classifier.fit(
    spanish_diagnostics_normalized_tokenized_vectorized["train"]["vectorized_text"],
    spanish_diagnostics_normalized_tokenized_vectorized["train"]["label"]
)

In [ ]:
print(sklearn.metrics.classification_report(
    spanish_diagnostics_normalized_tokenized_vectorized["test"]["label"],
    classifier.predict(spanish_diagnostics_normalized_tokenized_vectorized["test"]["vectorized_text"])
))

### Comparación de Word Embeddings

Realizamos el mismo proceso de vectorización pero utilizando las representaciones de dominio general y verificamos que el rendimiento es mucho menor al anterior.

In [ ]:
spanish_diagnostics_normalized_tokenized_vectorized_sbwce = spanish_diagnostics_normalized_tokenized.map(
    lambda x: {
        "vectorized_text" : to_vector(x["tokenized_text"],sbwce)
    })

In [ ]:
examples_3d_sbwce = projector.fit_transform(spanish_diagnostics_normalized_tokenized_vectorized_sbwce["test"]["vectorized_text"])
labels = spanish_diagnostics_normalized_tokenized_vectorized["test"].features["label"].int2str(spanish_diagnostics_normalized_tokenized_vectorized["test"]["label"])

plot_3d(examples_3d_sbwce , labels)

In [ ]:
classifier_sbwce = sklearn.linear_model.LogisticRegression(solver='liblinear')

In [ ]:
classifier.fit(
    spanish_diagnostics_normalized_tokenized_vectorized_sbwce["train"]["vectorized_text"],
    spanish_diagnostics_normalized_tokenized_vectorized_sbwce["train"]["label"]
)

In [ ]:
print(sklearn.metrics.classification_report(
    spanish_diagnostics_normalized_tokenized_vectorized_sbwce["test"]["label"],
    classifier.predict(spanish_diagnostics_normalized_tokenized_vectorized_sbwce["test"]["vectorized_text"])
))